<a href="https://colab.research.google.com/github/threegenie/churn_project/blob/main/telco_customer_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **이 데이터를 선택한 이유 ?**
캐글에서 다양한 데이터를 찾던 중, 'Telco Customer Churn'이라는 데이터셋을 보게 되었습니다. 

최근에 핸드폰을 사용하면서 요금제 관련 불편한 점 때문에 기존에 오래 사용하던 통신사가 아닌 다른 통신사의 요금제로 바꾼 경험이 있었기 때문에, 관심이 생기는 주제였습니다. 

사람들이 통신사를 바꾸게 될 때 단순히 불편해서만이 아닌, 보편적인 이유가 있을지 궁금해져서 여러 특성들을 통해 통신사 변경 여부를 예측하는 모델을 만들어 보려고 합니다.

### 통신사 이탈 여부 예측 모델


### **특성 설명**

* Gender : 성별 

* SeniorCitizen : 고령 여부 

* Partner : 배우자 유무 

* Dependents : 부양가족 유무 

* Tenure : 직장 근속 개월수 

* PhoneService : 전화 서비스 여부 

* MultipleLines : 다중 회선 여부

* InternetService : 인터넷 공급자 
  *  DSL
  *  Fiber optic

* OnlineSecurity : 온라인 보안 서비스 이용 여부

* OnlineBackup : 온라인 백업 서비스 이용 여부

* DeviceProtection : 기기보호 서비스 이용 여부

* TechSupport : 기술지원 서비스 이용 여부 

* StreamingTV : TV채널 스트리밍 이용 여부

* StreamingMovies : 영화 스트리밍 이용 여부 

* Contract : 계약 기간 
  * Month-to-Month 
  * One year 
  * Two year

* PaperlessBilling : 종이 고지서 신청 여부

* PaymentMethod : 요금 지불 방법
  * Electronic check
  * Bank transfer (automatic)
  * Mailed check
  * Credit card (automatic)

* MonthlyCharges : 월별 요금 

* TotalCharges : 총 요금 

* Churn : 이용자 서비스 해지 여부

In [ ]:
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pandas-profiling==2.*
    !pip install pdpbox
    !pip install shap

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Telco-Customer-Churn.csv')
df = df.drop('customerID',axis=1)
df.head()

예측할 특성은 Churn, 즉 서비스 이용자가 통신사를 변경했는지의 여부입니다. 타겟의 특성이 yes/no로 이루어져 있기 때문에, 분류 모델을 이용해 이 문제를 해결할 것입니다.

모든 분류 건수 중에서 분류기가 몇 개의 정답을 맞췄는지의 여부를 파악할 필요가 있기 때문에 Accuracy를 사용할 것입니다. 그리고 Accuracy의 단점을 보완하기 위해 추가로 recall과 F1 Score를 사용하여 모델의 성능을 평가하려고 합니다.

시계열 데이터가 아니기 때문에, 무작위로 데이터를 분리하여 훈련/검증/테스트 데이터셋을 만드는 방법을 택할 것입니다.

In [ ]:
def processing(df):
  #TotalCharges - 숫자형인데 object로 표시되는 데이터를 numeric화
  df['TotalCharges'] = df['TotalCharges'].str.replace('.','')
  df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors='coerce')

  #Contract - 문자로 되어있는 계약 기간을 바꾸기
  mapping = {'Month-to-month':0,'One year':1,'Two year':2}
  df['Contract'] = df['Contract'].replace(mapping)

  #MultipleLines - yes / no / no phone service를 숫자로 바꾸기
  mapping2 = {'No phone service':0,'No':1,'Yes':2}
  df['MultipleLines'] = df['MultipleLines'].replace(mapping2)

  #그 외 yes/no/no internet service를 항목으로 가지는 특성들 숫자로 바꾸기
  mapping3 = {'No internet service':0,'No':1,'Yes':2}
  no_internet_feature = ['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']
  df[no_internet_feature] = df[no_internet_feature].replace(mapping3)

  #Chrun - yes/no를 숫자로 바꾸기
  mapping4 = {'No':0,'Yes':1}
  df['Churn'] = df['Churn'].replace(mapping4)

  #PaymentMethod - 간단하게 바꾸기
  mapping5 = {'Bank transfer (automatic)':'Bank transfer','Credit card (automatic)':'Credit card'}
  df['PaymentMethod'] = df['PaymentMethod'].replace(mapping5)

processing(df)

**만들 수 있는 특성**
1. 배우자와 부양가족이 모두 있는지 여부
2. 온라인 보안, 백업 서비스를 모두 사용하는지 여부
3. tv채널과 영화 스트리밍 서비스를 모두 이용하는지 여부
4. 계약 기간별 월별 요금

In [ ]:
def features(df):
  #배우자와 부양가족이 모두 있는지 여부 - Yes : 1, No : 0
  df['Family'] = np.where((df['Partner']==1) & (df['Dependents']==1),1,0)

  #온라인 보안, 백업 서비스를 모두 이용하는지 여부
  df['OnlineService'] = np.where((df['OnlineSecurity']==2) & (df['OnlineBackup']==2),1,0)

  #tv채널과 영화 스트리밍 서비스를 모두 이용하는지 여부
  df['StreamingService'] = np.where((df['StreamingTV']==2) & (df['StreamingMovies']==2),1,0)

  #온라인, 스트리밍 서비스를 모두 이용하는지 여부
  df['FullService'] = np.where((df['OnlineService']==1) & (df['StreamingService']==1),1,0)

  #보험성 서비스를 몇개나 이용하는지
  df['security'] = np.where((df['OnlineSecurity']==2),1,0)
  df['backup'] = np.where(df['OnlineBackup']==2,1,0)
  df['protect'] = np.where(df['DeviceProtection']==2,1,0)
  df['support'] = np.where(df['TechSupport']==2,1,0)
  df['InsuranceServices'] = df['security']+df['backup']+df['protect']+df['support']
  df.drop(['security','backup','protect','support'],axis=1)

features(df)
df.head()

> 고객당 월별 요금이 가장 높은 지불 방법

In [ ]:
monthly_charges_by_pay_method = df.groupby(['PaymentMethod']).mean()[['MonthlyCharges']].reset_index()
monthly_charges_by_pay_method.head()

In [ ]:
plt.figure(figsize = (10, 6))
sns.barplot(x='PaymentMethod',y='MonthlyCharges',data=monthly_charges_by_pay_method, palette='hls')
plt.title('Monthly Charges by Payment Method')

> 월별 요금에 따른 통신사 이탈 여부

In [ ]:
sns.boxplot(
    x = 'Churn',
    y = 'MonthlyCharges',
    data = df,
    palette='deep'
)
plt.title('Customer Churn by Monthly Charges')

> 전체 요금에 따른 통신사 이탈 여부

In [ ]:
sns.boxplot(
    x = 'Churn',
    y = 'TotalCharges',
    data = df,
    palette='deep'
)
plt.title('Customer Churn by Monthly Charges')

> 각 특성별 고객들의 통신사 이탈 비율

In [ ]:
categorical_vars=['gender', 'SeniorCitizen','PhoneService', 'MultipleLines', 
            'InternetService', 'DeviceProtection', 'TechSupport', 'Contract',
            'PaperlessBilling', 'PaymentMethod','Family','OnlineService','StreamingService','FullService','InsuranceServices']
num_plots = len(categorical_vars)
total_cols = 4
total_rows = num_plots//total_cols + 1
fig, axs = plt.subplots(nrows=total_rows, ncols=total_cols,
                        figsize=(7*total_cols, 7*total_rows), constrained_layout=True)
for i, var in enumerate(categorical_vars):
    row = i//total_cols
    pos = i % total_cols
    plot = sns.countplot(x=var, data=df,palette = 'husl',
                         hue = "Churn",
                         ax=axs[row][pos])

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df, test_size=0.1, random_state=2)
train, test = train_test_split(train, test_size=0.05, random_state=2)
train.shape, val.shape, test.shape

In [ ]:
target = 'Churn'
features = ['gender', 'SeniorCitizen', 'tenure','PhoneService', 'MultipleLines', 
            'InternetService', 'DeviceProtection', 'TechSupport', 'Contract',
            'PaperlessBilling', 'PaymentMethod','MonthlyCharges', 'TotalCharges',
            'Family','OnlineService','StreamingService','FullService','InsuranceServices']

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

> 타겟 특성의 비율을 알아보고, 가중치 부여하기

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
#가중치
ratio = 0.26/0.73
ratio





> Baseline




In [ ]:
from category_encoders import OrdinalEncoder
from category_encoders import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import sklearn.metrics as metrics
import lightgbm as lgb
from lightgbm import LGBMClassifier

pipe = Pipeline([
    ('preprocessing', make_pipeline(OrdinalEncoder(), SimpleImputer())),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=2, n_jobs=-1)) 
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_val)

print('Accuracy: ', metrics.accuracy_score(y_val, y_pred))
print('Recall: ',metrics.recall_score(y_val,y_pred,average='binary'))
print('F1 Score: ', metrics.f1_score(y_val, y_pred, average="binary"))



> permutation importance를 통한 feature 선택



In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
    pipe.named_steps['rf'],
    scoring='accuracy',
    n_iter=5, 
    random_state=2
)

X_val_transformed = pipe.named_steps['preprocessing'].transform(X_val)
permuter.fit(X_val_transformed, y_val);

eli5.show_weights(
    permuter, 
    top=None,
    feature_names=features 
)

In [ ]:
minimum_importance = 0.001
mask = permuter.feature_importances_ > minimum_importance
feature = X_train.columns[mask]
X_train_selected = X_train[feature]
X_val_selected = X_val[feature]

In [ ]:
feature

In [ ]:
X_train = train[feature]
y_train = train[target]
X_val = val[feature]
y_val = val[target]
X_test = test[feature]
y_test = test[target]



> hyperparameter tuning



RandomForest Model

In [ ]:
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

param_distributions = { 
    'simpleimputer__strategy':['mean','median','most_frequent'],
    "randomforestclassifier__n_estimators": list(range(10, 1000, 100)),
    "randomforestclassifier__max_depth": list(range(2,10,2)),
    "randomforestclassifier__max_features": list(range(3,12,3)),
    "randomforestclassifier__min_samples_split": list(range(3,9,2))
}

model = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier()
)

search = RandomizedSearchCV(
    model, 
    param_distributions=param_distributions, 
    n_iter=50, 
    cv=3, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1, 
    random_state=2,
)

search.fit(X_train, y_train);
model = search.best_estimator_
y_val_pred = model.predict(X_val)

print('\n최적 하이퍼파라미터: ', search.best_params_)
print('Accuracy: ', metrics.accuracy_score(y_val, y_val_pred))
print('Recall: ',metrics.recall_score(y_val,y_pred,average='binary'))
print('F1 score: ', metrics.f1_score(y_val, y_val_pred, average='binary'))

LGBM model

In [ ]:
param_distributions = { 
    "lgbmclassifier__n_estimators": list(range(10, 1000, 100)),
    "lgbmclassifier__max_depth": list(range(2,10,2)),
    "lgbmclassifier__max_features": list(range(3,12,3)),
    "lgbmclassifier__min_samples_split": list(range(3,9,2)),
    "lgbmclassifier__num_iterations":list(range(100,1000,100))

}

model_lgbm = make_pipeline(
    OrdinalEncoder(), 
    LGBMClassifier(scale_pos_weight = ratio,learning_rate=0.1)
)

search_lgbm = RandomizedSearchCV(
    model_lgbm, 
    param_distributions=param_distributions, 
    n_iter=50, 
    cv=3, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1, 
    random_state=2
)

search_lgbm.fit(X_train, y_train);
model_lgbm = search_lgbm.best_estimator_
y_val_pred_lgbm = model_lgbm.predict(X_val)

print('\n최적 하이퍼파라미터: ', search_lgbm.best_params_)
print('Accuracy: ', metrics.accuracy_score(y_val, y_val_pred_lgbm))
print('Recall: ',metrics.recall_score(y_val,y_val_pred_lgbm,average='binary'))
print('F1 score: ', metrics.f1_score(y_val, y_val_pred_lgbm, average='binary'))

XGBClassifier model

In [ ]:
from xgboost import XGBClassifier

param_distributions = { 
    "xgbclassifier__n_estimators": list(range(10, 1000, 100)),
    "xgbclassifier__max_depth": list(range(2,10,2)),
    "xgbclassifier__max_features": list(range(3,12,3)),
    "xgbclassifier__min_samples_split": list(range(3,9,2)),
    "xgbclassifier__num_iterations":list(range(100,1000,100))

}

model_xgb = make_pipeline(
    OrdinalEncoder(), 
    XGBClassifier(scale_pos_weight = ratio,learning_rate=0.1)
)

search_xgb = RandomizedSearchCV(
    model_xgb, 
    param_distributions=param_distributions, 
    n_iter=50, 
    cv=3, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1, 
    random_state=2
)

search_xgb.fit(X_train, y_train);
model_xgb = search_xgb.best_estimator_
y_val_pred_xgb = model_xgb.predict(X_val)

print('\n최적 하이퍼파라미터: ', search_xgb.best_params_)
print('Accuracy: ', metrics.accuracy_score(y_val, y_val_pred_xgb))
print('Recall: ',metrics.recall_score(y_val,y_val_pred_xgb,average='binary'))
print('F1 score: ', metrics.f1_score(y_val, y_val_pred_xgb, average='binary'))



> ROC Curve, AUC Score



In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

y_pred_proba = model_xgb.predict_proba(X_val)[:,1]
fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

roc = pd.DataFrame({
    'FPR(Fall-out)': fpr, 
    'TPRate(Recall)': tpr, 
    'Threshold': thresholds
})
plt.scatter(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('FPR(Fall-out)')
plt.ylabel('TPR(Recall)')

In [ ]:
auc_score = roc_auc_score(y_val, y_pred_proba)
auc_score



>  PDP 



**tenure** 특성

In [ ]:
tenure = 'tenure'

In [ ]:
from pdpbox.pdp import pdp_isolate, pdp_plot

tenure = 'tenure'
isolated = pdp_isolate(
    model=model_xgb,
    dataset=X_val, 
    model_features=X_val.columns, 
    feature=tenure,

)
pdp_plot(isolated, feature_name=tenure);


**Contract** 특성

In [ ]:
contract = 'Contract'
isolated2 = pdp_isolate(
    model=model_xgb,
    dataset=X_val, 
    model_features=X_val.columns, 
    feature=contract,

)
pdp_plot(isolated2, feature_name=contract);


**MultipleLines** 특성

In [ ]:
line = 'MultipleLines'

isolated3 = pdp_isolate(
    model=model_xgb,
    dataset=X_val, 
    model_features=X_val.columns, 
    feature=line,

)
pdp_plot(isolated3, feature_name=line);




> SHAP






In [ ]:
processor = make_pipeline(
    OrdinalEncoder(), 
)

X_train_processed = processor.fit_transform(X_train)
X_val_processed = processor.transform(X_val)

eval_set = [(X_train_processed, y_train), 
            (X_val_processed, y_val)]

model_shap = XGBClassifier(
    n_estimators=310,min_samples_split=3,max_features=6,max_depth=8,num_iterations=300,
    random_state=2,verbose=10, n_jobs=-1, scale_pos_weight=ratio,cv=3,
    scoring='accuracy',return_train_score=True)

model_shap.fit(X_train_processed, y_train, eval_set=eval_set, eval_metric='auc', 
          early_stopping_rounds=10)



> Force plot



case 1

In [ ]:
row = X_train_processed.iloc[[1]]
row

In [ ]:
import shap

explainer = shap.TreeExplainer(model_shap)
shap_values = explainer.shap_values(row)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values,
    features=row
)

case 2

In [ ]:
row2 = X_train_processed.iloc[[5]]
row2

In [ ]:
explainer = shap.TreeExplainer(model_shap)
shap_values = explainer.shap_values(row2)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values,
    features=row2
)


> Summary plot : scatter, violin, bar



In [ ]:
shap.initjs()
rows = X_train_processed
shap_values = explainer.shap_values(rows)
shap.summary_plot(shap_values, rows)

In [ ]:
shap.initjs()
shap.summary_plot(shap_values, rows, plot_type='violin')

In [ ]:
shap.initjs()
shap.summary_plot(shap_values, rows,plot_type='bar')